# Chatbot Agent Testing Notebook

This notebook provides a user-friendly interface for testing the Chatbot Agent with pre-generated questions.
It's designed for users with little programming experience.

## Features:
- Load and edit agent configuration settings
- Load test questions from JSON files
- Process questions through the Chatbot Agent
- Review and analyze results
- Export results with timestamps

## Getting Started:
1. Run cells in order from top to bottom
2. Edit configuration values as needed
3. Load test questions from file (generated using question_generator.ipynb)
4. Review questions before processing
5. Run the agent and review results

## Question Generation:
Use the separate `question_generator.ipynb` notebook to create test questions, then load them here.

In [1]:
# Import required libraries
import json
import os
import sys
from datetime import datetime
from pathlib import Path

import ipywidgets as widgets
import pandas as pd
import yaml
from IPython.display import display, HTML, clear_output

# Set the working directory to the root of the project
os.chdir('/workspace')

# Add workspace to path for imports (this helps with relative imports)
sys.path.insert(0, '/workspace')

# Import our system components
from src.nodes.chatbot_agent import ChatbotAgentNode
from src.core.config.agent_config_manager import AgentConfigManager
from src.integrations.llm_providers import LLMProviderFactory

print("✅ All libraries imported successfully!")
print("Ready to start testing the Chatbot Agent.")

✅ All libraries imported successfully!
Ready to start testing the Chatbot Agent.


## Step 1: Load Configuration Settings

The following cell loads the current configuration for the Chatbot Agent.
You can edit these values to customize the agent's behavior.

In [2]:
# Load configuration from files and create temporary editable copies
config_base_path = Path('/workspace/config')
agent_config_path = config_base_path / 'agents' / 'chatbot_agent'
temp_config_dir = Path('/tmp/chatbot_agent_configs')

def load_and_create_temp_configs():
    """Load all configuration files and create temporary editable copies"""
    configs = {}
    
    # Create temp directory
    temp_config_dir.mkdir(exist_ok=True)
    
    # Load and copy agent config
    with open(agent_config_path / 'config.yaml', 'r') as f:
        configs['agent'] = yaml.safe_load(f)
    
    # Load and copy prompts
    with open(agent_config_path / 'prompts.yaml', 'r') as f:
        configs['prompts'] = yaml.safe_load(f)
    
    # Load and copy models
    with open(agent_config_path / 'models.yaml', 'r') as f:
        configs['models'] = yaml.safe_load(f)
    
    # Load shared models for reference
    with open(config_base_path / 'shared' / 'models.yaml', 'r') as f:
        configs['shared_models'] = yaml.safe_load(f)
    
    # Save temporary copies for editing
    temp_agent_path = temp_config_dir / 'config.yaml'
    temp_prompts_path = temp_config_dir / 'prompts.yaml'
    temp_models_path = temp_config_dir / 'models.yaml'
    
    with open(temp_agent_path, 'w') as f:
        yaml.dump(configs['agent'], f, default_flow_style=False, sort_keys=False)
    
    with open(temp_prompts_path, 'w') as f:
        yaml.dump(configs['prompts'], f, default_flow_style=False, sort_keys=False)
    
    with open(temp_models_path, 'w') as f:
        yaml.dump(configs['models'], f, default_flow_style=False, sort_keys=False)
    
    return configs, {
        'agent_config': temp_agent_path,
        'prompts_config': temp_prompts_path,
        'models_config': temp_models_path
    }

# Load configurations and create temp files
configs, temp_file_paths = load_and_create_temp_configs()

print("📁 Configuration files loaded and temporary copies created!")
print(f"Agent name: {configs['agent']['agent']['name']}")
print(f"Agent version: {configs['agent']['agent']['version']}")
print(f"Preferred model: {configs['agent']['models']['preferred']}")
print(f"\n💾 Temporary config files created at:")
for config_type, path in temp_file_paths.items():
    print(f"  {config_type}: {path}")
print(f"\n💡 These temp files can be edited directly and will be used in Step 5 if modified.")

📁 Configuration files loaded and temporary copies created!
Agent name: chatbot_agent
Agent version: 1.0.0
Preferred model: anthropic_general_budget

💾 Temporary config files created at:
  agent_config: /tmp/chatbot_agent_configs/config.yaml
  prompts_config: /tmp/chatbot_agent_configs/prompts.yaml
  models_config: /tmp/chatbot_agent_configs/models.yaml

💡 These temp files can be edited directly and will be used in Step 5 if modified.


## Step 2: Editable Configuration Settings

Edit these settings to customize how the Chatbot Agent behaves.
These variables map directly to the configuration files and can be exported later.

In [3]:
# Display and edit configuration files in separate windows

def load_config_file_contents():
    """Load current config file contents from temp files"""
    with open(temp_file_paths['agent_config'], 'r') as f:
        agent_config_content = f.read()
    with open(temp_file_paths['prompts_config'], 'r') as f:
        prompts_config_content = f.read()
    with open(temp_file_paths['models_config'], 'r') as f:
        models_config_content = f.read()
    
    return agent_config_content, prompts_config_content, models_config_content

# Load current config file contents
agent_config_content, prompts_config_content, models_config_content = load_config_file_contents()

print("⚙️ Configuration File Editor")
print("Edit the YAML configuration files below and use the Save buttons to apply changes.")
print("Changes are saved to temporary files and will be used in Step 5.\n")

# Create text areas for each config file
print("📄 1. Agent Configuration (config.yaml)")
print("Contains: agent settings, behavior, escalation thresholds")

agent_config_editor = widgets.Textarea(
    value=agent_config_content,
    description="",
    layout=widgets.Layout(width='100%', height='250px'),
    style={'description_width': '0px'}
)

def save_agent_config(button):
    """Save agent config changes"""
    try:
        # Validate YAML syntax
        yaml.safe_load(agent_config_editor.value)
        
        # Save to temp file
        with open(temp_file_paths['agent_config'], 'w') as f:
            f.write(agent_config_editor.value)
        
        print("✅ Agent config saved successfully!")
        
    except yaml.YAMLError as e:
        print(f"❌ YAML syntax error in agent config: {e}")
    except Exception as e:
        print(f"❌ Error saving agent config: {e}")

agent_save_btn = widgets.Button(description="Save Agent Config", button_style='success')
agent_save_btn.on_click(save_agent_config)

display(agent_config_editor)
display(agent_save_btn)

print("\n📄 2. Prompts Configuration (prompts.yaml)")
print("Contains: system prompts, response guidelines, communication style")

prompts_config_editor = widgets.Textarea(
    value=prompts_config_content,
    description="",
    layout=widgets.Layout(width='100%', height='250px'),
    style={'description_width': '0px'}
)

def save_prompts_config(button):
    """Save prompts config changes"""
    try:
        # Validate YAML syntax
        yaml.safe_load(prompts_config_editor.value)
        
        # Save to temp file
        with open(temp_file_paths['prompts_config'], 'w') as f:
            f.write(prompts_config_editor.value)
        
        print("✅ Prompts config saved successfully!")
        
    except yaml.YAMLError as e:
        print(f"❌ YAML syntax error in prompts config: {e}")
    except Exception as e:
        print(f"❌ Error saving prompts config: {e}")

prompts_save_btn = widgets.Button(description="Save Prompts Config", button_style='success')
prompts_save_btn.on_click(save_prompts_config)

display(prompts_config_editor)
display(prompts_save_btn)

print("\n📄 3. Models Configuration (models.yaml)")
print("Contains: preferred model, fallback models, model-specific settings")

# Show available model aliases from shared models config
def display_available_models():
    """Display available model aliases and their actual models"""
    try:
        shared_models = configs['shared_models']
        
        # Extract model aliases and models sections
        model_aliases = shared_models.get('model_aliases', {})
        models = shared_models.get('models', {})
        
        if not model_aliases:
            print("❌ No model aliases found in shared configuration")
            return
            
        print("\n🔍 Available Model Aliases:")
        print("Use these aliases in your models configuration below:\n")
        
        # Group by provider for better organization
        providers = {}
        for alias, actual_model_name in model_aliases.items():
            # Get model details from models section
            model_details = models.get(actual_model_name, {})
            provider = model_details.get('type', 'unknown')
            description = model_details.get('description', '')
            
            if provider not in providers:
                providers[provider] = []
            providers[provider].append({
                'alias': alias,
                'model_name': actual_model_name,
                'description': description
            })
        
        # Display by provider
        for provider, provider_models in providers.items():
            print(f"📡 {provider.upper()} Provider:")
            for model in provider_models:
                desc = f" - {model['description']}" if model['description'] else ""
                print(f"  • {model['alias']} → {model['model_name']}{desc}")
            print()
        
        # Show current configuration
        try:
            current_models_config = yaml.safe_load(models_config_content)
            current_preferred = current_models_config.get('preferred', 'unknown')
            current_fallback = current_models_config.get('fallback', [])
        except:
            current_preferred = 'unknown'
            current_fallback = []
        
        print(f"📋 Current Models Configuration:")
        print(f"  Preferred: {current_preferred}")
        print(f"  Fallback: {current_fallback}")
        print()
        
        print("💡 Tips for editing:")
        print("  • Change 'preferred' to any alias from the list above")
        print("  • Add/remove aliases in the 'fallback' list")
        print("  • Aliases are case-sensitive")
        print("  • Invalid aliases will cause errors during processing")
        print()
        
    except Exception as e:
        print(f"❌ Error loading available models: {e}")
        print("Continuing with models configuration editor...\n")

# Display available models before showing the editor
display_available_models()

models_config_editor = widgets.Textarea(
    value=models_config_content,
    description="",
    layout=widgets.Layout(width='100%', height='200px'),
    style={'description_width': '0px'}
)

def save_models_config(button):
    """Save models config changes"""
    try:
        # Validate YAML syntax
        parsed_config = yaml.safe_load(models_config_editor.value)
        
        # Additional validation for model aliases
        if isinstance(parsed_config, dict):
            preferred = parsed_config.get('preferred')
            fallback = parsed_config.get('fallback', [])
            
            # Get available aliases
            model_aliases = configs['shared_models'].get('model_aliases', {})
            
            # Check if preferred model exists
            if preferred and preferred not in model_aliases:
                print(f"⚠️ Warning: Preferred model '{preferred}' not found in available model aliases")
            
            # Check fallback models
            for fb_model in fallback:
                if fb_model not in model_aliases:
                    print(f"⚠️ Warning: Fallback model '{fb_model}' not found in available model aliases")
        
        # Save to temp file
        with open(temp_file_paths['models_config'], 'w') as f:
            f.write(models_config_editor.value)
        
        print("✅ Models config saved successfully!")
        
    except yaml.YAMLError as e:
        print(f"❌ YAML syntax error in models config: {e}")
    except Exception as e:
        print(f"❌ Error saving models config: {e}")

models_save_btn = widgets.Button(description="Save Models Config", button_style='success')
models_save_btn.on_click(save_models_config)

display(models_config_editor)
display(models_save_btn)

# Save All button for convenience
def save_all_configs(button):
    """Save all config changes at once"""
    save_agent_config(None)
    save_prompts_config(None)
    save_models_config(None)

print("\n💾 Save All Changes")
save_all_btn = widgets.Button(description="Save All Configs", button_style='info')
save_all_btn.on_click(save_all_configs)
display(save_all_btn)

print(f"\n💾 Temp config files location:")
for config_type, path in temp_file_paths.items():
    print(f"  {config_type}: {path}")

print("\n💡 Tips:")
print("  • Edit YAML directly in the text areas above")
print("  • Use Save buttons to apply changes to temp files")
print("  • YAML syntax is validated before saving")
print("  • Model aliases are validated against available models")
print("  • Changes will be used in Step 5 when processing questions")
print("  • Original config files remain unchanged")

⚙️ Configuration File Editor
Edit the YAML configuration files below and use the Save buttons to apply changes.
Changes are saved to temporary files and will be used in Step 5.

📄 1. Agent Configuration (config.yaml)
Contains: agent settings, behavior, escalation thresholds


Textarea(value="agent:\n  name: chatbot_agent\n  version: 1.0.0\n  description: Primary response generation ag…

Button(button_style='success', description='Save Agent Config', style=ButtonStyle())


📄 2. Prompts Configuration (prompts.yaml)
Contains: system prompts, response guidelines, communication style


Textarea(value="system: 'You are a professional customer service chatbot dedicated to providing exceptional\n …

...

Textarea(value='primary_model: local_general_standard\nmodel_preferences:\n  general_queries:\n    primary: an…

Button(button_style='success', description='Save Models Config', style=ButtonStyle())


💾 Save All Changes


Button(button_style='info', description='Save All Configs', style=ButtonStyle())


💾 Temp config files location:
  agent_config: /tmp/chatbot_agent_configs/config.yaml
  prompts_config: /tmp/chatbot_agent_configs/prompts.yaml
  models_config: /tmp/chatbot_agent_configs/models.yaml

💡 Tips:
  • Edit YAML directly in the text areas above
  • Use Save buttons to apply changes to temp files
  • YAML syntax is validated before saving
  • Model aliases are validated against available models
  • Changes will be used in Step 5 when processing questions
  • Original config files remain unchanged


## Step 3: Load Test Questions

Load test questions from a JSON file. Use question_generator.ipynb to create new questions.

In [4]:
# File upload widget for loading questions
file_upload = widgets.FileUpload(
    accept='.json',
    multiple=False,
    description='Upload questions file:'
)

# Instructions for file format
print("📝 Load Test Questions")
print("\n💡 How to get test questions:")
print("1. Use question_generator.ipynb to create test questions")
print("2. Upload the generated JSON file below")
print("3. Supported formats: Full export or questions-only JSON")
print("\n📄 Expected JSON format:")
print("- Full export: {'metadata': {...}, 'questions': [...]}")
print("- Questions only: [{'id': 1, 'question': '...', 'customer_type': '...', 'complexity': '...'}]")
print("\n📁 Upload your questions file:")
display(file_upload)

📝 Load Test Questions

💡 How to get test questions:
1. Use question_generator.ipynb to create test questions
2. Upload the generated JSON file below
3. Supported formats: Full export or questions-only JSON

📄 Expected JSON format:
- Full export: {'metadata': {...}, 'questions': [...]}
- Questions only: [{'id': 1, 'question': '...', 'customer_type': '...', 'complexity': '...'}]

📁 Upload your questions file:


FileUpload(value=(), accept='.json', description='Upload questions file:')

In [5]:
# Load test questions from uploaded file
test_questions = []

def load_questions_from_file(file_content, filename):
    """Load questions from uploaded JSON file"""
    try:
        # Handle different content types
        if isinstance(file_content, memoryview):
            # Convert memoryview to bytes
            content_bytes = file_content.tobytes()
        elif hasattr(file_content, 'decode'):
            # Already bytes
            content_bytes = file_content
        else:
            # Convert to bytes if it's a string or other type
            content_bytes = str(file_content).encode('utf-8')
        
        # Decode to string and parse JSON
        data = json.loads(content_bytes.decode('utf-8'))
        
        # Handle different JSON formats
        if isinstance(data, dict):
            # Full export format with metadata
            if 'questions' in data:
                questions = data['questions']
                metadata = data.get('metadata', {})
                print(f"📄 Loaded file with metadata:")
                print(f"  Generation model: {metadata.get('generation_model', 'unknown')}")
                print(f"  Generation timestamp: {metadata.get('generation_timestamp', 'unknown')}")
                print(f"  Question count: {metadata.get('question_count', len(questions))}")
                return questions
            else:
                # Single question object
                return [data]
        elif isinstance(data, list):
            # Questions-only format
            return data
        else:
            print(f"❌ Unexpected data format: {type(data)}")
            return []
            
    except json.JSONDecodeError as e:
        print(f"❌ JSON parsing error: {e}")
        return []
    except Exception as e:
        print(f"❌ Error loading file: {e}")
        print(f"   File content type: {type(file_content)}")
        return []

def validate_questions(questions):
    """Validate and normalize question format"""
    validated_questions = []
    
    for i, q in enumerate(questions):
        if isinstance(q, dict):
            # Ensure required fields exist
            validated_q = {
                "id": q.get("id", i + 1),
                "question": q.get("question", f"Question {i + 1}"),
                "customer_type": q.get("customer_type", "normal"),
                "complexity": q.get("complexity", "medium")
            }
            validated_questions.append(validated_q)
        else:
            # Convert string to dict if needed
            validated_q = {
                "id": i + 1,
                "question": str(q),
                "customer_type": "normal",
                "complexity": "medium"
            }
            validated_questions.append(validated_q)
    
    return validated_questions

# Process uploaded file
if file_upload.value:
    uploaded_file = None
    filename = None
    file_content = None
    
    # Handle different file upload widget formats
    if isinstance(file_upload.value, tuple) and len(file_upload.value) > 0:
        print("📋 Using tuple format")
        uploaded_file = file_upload.value[0]
        filename = uploaded_file['name']
        file_content = uploaded_file['content']
        print(f"🔍 File content type: {type(file_content)}")
    elif isinstance(file_upload.value, dict) and len(file_upload.value) > 0:
        print("📋 Using dict format")
        uploaded_file = list(file_upload.value.values())[0]
        filename = uploaded_file['metadata']['name']
        file_content = uploaded_file['content']
        print(f"🔍 File content type: {type(file_content)}")
    else:
        print(f"❌ Unable to read uploaded file format")
        print(f"   Type: {type(file_upload.value)}")
        print(f"   Length: {len(file_upload.value) if hasattr(file_upload.value, '__len__') else 'No length'}")
        print(f"   Content: {file_upload.value}")
    
    if uploaded_file and filename and file_content is not None:
        print(f"📁 Loading questions from: {filename}")
        
        raw_questions = load_questions_from_file(file_content, filename)
        
        if raw_questions:
            test_questions = validate_questions(raw_questions)
            print(f"✅ Loaded {len(test_questions)} test questions")
            
            # Display first few questions as preview
            print("\n📋 Preview of loaded questions:")
            for i, q in enumerate(test_questions[:3]):
                print(f"  {i+1}. {q['question']} [{q['customer_type']}]")
            if len(test_questions) > 3:
                print(f"  ... and {len(test_questions) - 3} more questions")
                
            # Show distribution
            df_preview = pd.DataFrame(test_questions)
            print("\n📊 Question Distribution:")
            print(f"  Customer types: {dict(df_preview['customer_type'].value_counts())}")
            print(f"  Complexities: {dict(df_preview['complexity'].value_counts())}")
            
        else:
            print("❌ No questions loaded from file")
    else:
        print("❌ Error accessing uploaded file")
        print(f"   uploaded_file: {uploaded_file is not None}")
        print(f"   filename: {filename}")
        print(f"   file_content: {file_content is not None}")
else:
    print("⚠️ Please upload a JSON file with test questions.")
    print("💡 Use question_generator.ipynb to create test questions first.")

📋 Using tuple format
🔍 File content type: <class 'memoryview'>
📁 Loading questions from: questions_only_20250718_1744.json
✅ Loaded 20 test questions

📋 Preview of loaded questions:
  1. Why did my premium increase by $200? This is ridiculous - I haven't had any claims! [frustrated]
  2. I just had a car accident. What do I need to do right now? [urgent]
  3. Can someone explain what a deductible is? I keep seeing this word but don't understand. [confused]
  ... and 17 more questions

📊 Question Distribution:
  Customer types: {'frustrated': np.int64(5), 'urgent': np.int64(5), 'confused': np.int64(5), 'normal': np.int64(5)}
  Complexities: {'medium': np.int64(7), 'simple': np.int64(7), 'complex': np.int64(6)}


## Step 4: Review and Edit Questions

Review the loaded questions and make any edits before processing.

In [6]:
# Create interactive question editor
if test_questions:
    print("📝 Question Editor - You can modify questions before processing")
    print("Edit the questions in the table below, then run the next cell to process them.\n")
    
    # Convert to DataFrame for easy editing
    df = pd.DataFrame(test_questions)
    
    # Ensure required columns exist
    if 'id' not in df.columns:
        df['id'] = range(1, len(df) + 1)
    if 'customer_type' not in df.columns:
        df['customer_type'] = 'normal'
    if 'complexity' not in df.columns:
        df['complexity'] = 'medium'
    
    # Display editable table
    print("Current questions (you can edit the JSON below if needed):")
    display(df)
    
    # Show JSON for manual editing if needed
    questions_json = widgets.Textarea(
        value=json.dumps(test_questions, indent=2),
        description="Questions JSON:",
        layout=widgets.Layout(width='100%', height='200px')
    )
    
    print("\nAdvanced: Edit questions as JSON (optional):")
    display(questions_json)
    
    def update_questions_from_json():
        """Update questions from the JSON editor"""
        global test_questions
        try:
            test_questions = json.loads(questions_json.value)
            print("✅ Questions updated from JSON editor")
        except Exception as e:
            print(f"❌ Error parsing JSON: {e}")
    
    # Button to update from JSON
    update_btn = widgets.Button(description="Update from JSON")
    update_btn.on_click(lambda b: update_questions_from_json())
    display(update_btn)
    
else:
    print("⚠️ No questions loaded. Please upload a questions file in the previous step.")

📝 Question Editor - You can modify questions before processing
Edit the questions in the table below, then run the next cell to process them.

Current questions (you can edit the JSON below if needed):


,id,question,customer_type,complexity
0,1,Why did my premium increase by $200? This is r...,frustrated,medium
1,2,I just had a car accident. What do I need to d...,urgent,simple
2,3,Can someone explain what a deductible is? I ke...,confused,simple
3,4,How do I add my teenage daughter to my auto po...,normal,simple
4,5,I've been a customer for 15 years and you deni...,frustrated,complex
5,6,What's the difference between comprehensive an...,confused,medium
6,7,My house just flooded - I need emergency assis...,urgent,complex
7,8,Can I get proof of insurance sent to my phone?...,urgent,simple
8,9,How do I update my billing information?,normal,simple
9,10,What exactly is an 'act of God' and why isn't ...,confused,medium



Advanced: Edit questions as JSON (optional):


Textarea(value='[\n  {\n    "id": 1,\n    "question": "Why did my premium increase by $200? This is ridiculous…

Button(description='Update from JSON', style=ButtonStyle())

## Step 5: Process Questions Through Chatbot Agent

Run the questions through the Chatbot Agent and collect results.

In [ ]:
# Process questions through the Chatbot Agent using temp configs if available
if not test_questions:
    print("⚠️ No questions to process. Please load or generate questions first.")
else:
    print(f"🤖 Processing {len(test_questions)} questions through Chatbot Agent...")
    
    # Check if temp config files exist and load them
    def load_config_for_processing():
        """Load config from temp files if they exist, otherwise use base configs"""
        if all(path.exists() for path in temp_file_paths.values()):
            print("📁 Using temporary config files with your edits...")
            
            # Load from temp files
            with open(temp_file_paths['agent_config'], 'r') as f:
                agent_config = yaml.safe_load(f)
            with open(temp_file_paths['prompts_config'], 'r') as f:
                prompts_config = yaml.safe_load(f)
            with open(temp_file_paths['models_config'], 'r') as f:
                models_config = yaml.safe_load(f)
            
            return agent_config, prompts_config, models_config, True
        else:
            print("📁 Using base config files...")
            return configs['agent'], configs['prompts'], configs['models'], False
    
    # Load configuration for processing
    current_agent_config, current_prompts_config, current_models_config, using_temp_files = load_config_for_processing()
    
    # Extract current values - FIXED: Use standardized model config structure
    if 'primary_model' in current_models_config:
        current_preferred_model = current_models_config['primary_model']
    elif 'preferred' in current_models_config:
        current_preferred_model = current_models_config['preferred']
    elif 'model_preferences' in current_models_config and current_models_config['model_preferences']:
        # Get first task preferences as fallback
        first_task = next(iter(current_models_config['model_preferences'].values()), {})
        current_preferred_model = first_task.get('primary', 'anthropic_general_budget')
    else:
        current_preferred_model = 'anthropic_general_budget'  # fallback
    
    current_temperature = current_agent_config['settings']['temperature']
    
    print(f"Using model: {current_preferred_model}")
    print(f"Temperature: {current_temperature}")
    print(f"Config source: {'Temporary files (with edits)' if using_temp_files else 'Base config files'}")
    print("\n" + "="*50)
    
    def analyze_chatbot_response(response: str, confidence: float) -> dict:
        """
        Analyze chatbot response to determine escalation and input needs
        """
        response_lower = response.lower()
        
        # Escalation indicators - phrases that suggest referring to human/department
        escalation_phrases = [
            "let me transfer you to",
            "i'll connect you with",
            "please contact our",
            "speak with a specialist",
            "escalate this to",
            "connect you with the",
            "transfer to our",
            "speak with someone from",
            "you'll need to contact",
            "reach out to our",
            "i recommend speaking with",
            "forward this to"
        ]
        
        # More input indicators - phrases requesting additional information
        input_request_phrases = [
            "could you provide",
            "can you tell me more about",
            "what is your",
            "to better assist you",
            "i need more information",
            "could you clarify",
            "can you specify",
            "what type of",
            "please let me know",
            "can you share",
            "what specific",
            "which policy"
                    ]
        
        # Question indicators (asking user for info to continue conversation)
        question_patterns = [
            "could you",
            "can you",
            "would you mind",
            "please provide",
            "do you have",
            "what is",
            "which",
            "when did",
            "?"
        ]
        
        # Check for escalation intent
        escalation_detected = any(phrase in response_lower for phrase in escalation_phrases)
        
        # Check for information requests
        input_needed = any(phrase in response_lower for phrase in input_request_phrases)
        
        # Check if response is primarily asking questions to continue conversation
        question_count = response.count('?')
        is_asking_questions = question_count > 0 and any(pattern in response_lower for pattern in question_patterns)
        
        # Confidence-based escalation (very low confidence = likely needs human help)
        low_confidence_escalation = confidence < 0.6
        
        # Final determination with priority logic
        needs_escalation = escalation_detected or low_confidence_escalation
        # Only flag needs_more_input if NOT escalating (escalation takes priority)
        needs_more_input = (input_needed or is_asking_questions) and not needs_escalation
        
        return {
            'needs_escalation': needs_escalation,
            'needs_more_input': needs_more_input,
            'escalation_reason': 'explicit_transfer' if escalation_detected else 'low_confidence' if low_confidence_escalation else None,
            'input_reason': 'information_request' if input_needed else 'clarifying_questions' if is_asking_questions else None,
            'question_count': question_count,
            'confidence_score': confidence,
            'analysis_details': {
                'escalation_detected': escalation_detected,
                'input_needed': input_needed,
                'is_asking_questions': is_asking_questions,
                'low_confidence': low_confidence_escalation
            }
        }
    
    # Create a simple mock context provider for testing
    class MockContextProvider:
        """Simple mock context provider for testing"""
        
        def get_context_summary(self, user_id: str, session_id: str) -> dict:
            """Return a simple mock context summary"""
            return {
                'entries_count': 0,
                'type_breakdown': {},
                'recent_queries': [],
                'escalation_count': 0,
                'last_activity': None
            }
        
        def save_context_entry(self, entry) -> bool:
            """Mock save method"""
            return True
        
        def get_recent_context(self, user_id: str, session_id: str, limit: int = 10) -> list:
            """Mock recent context method"""
            return []
    
    # Initialize the Chatbot Agent
    try:
        # Create config manager with the correct config directory
        config_manager = AgentConfigManager(config_dir='/workspace/config')
        
        # Override config if using temp files - FIXED: Use correct attributes
        if using_temp_files:
            # Create a temporary AgentConfig object with our custom configs
            from src.core.config.agent_config_manager import AgentConfig
            
            # Create custom agent config object
            agent_section = current_agent_config.get('agent', {})
            custom_agent_config = AgentConfig(
                name=agent_section.get('name', 'chatbot_agent'),
                description=agent_section.get('description', ''),
                type=agent_section.get('type', 'agent'),
                version=agent_section.get('version', '1.0.0'),
                models=current_models_config,
                settings=current_agent_config.get('settings', {}),
                prompts=current_prompts_config,
                behavior=current_agent_config.get('behavior', {}),
                escalation=current_agent_config.get('escalation', {}),
                evaluation=current_agent_config.get('evaluation', {}),
                routing=current_agent_config.get('routing', {})
            )
            
            # Override the agent config in the manager
            config_manager._agents['chatbot_agent'] = custom_agent_config
            print("✅ Overrode config manager with temporary configs")
        
        # Create a mock context provider for testing
        context_provider = MockContextProvider()
        
        # Initialize Chatbot Agent
        chatbot_agent = ChatbotAgentNode(config_manager, context_provider)
        
        print("✅ Chatbot Agent initialized successfully")
        
    except Exception as e:
        print(f"❌ Error initializing Chatbot Agent: {e}")
        print("Continuing with mock responses for demonstration...")
        chatbot_agent = None
    
    # Process each question
    results = []
    
    for i, question_data in enumerate(test_questions):
        question_id = question_data.get('id', i + 1)
        question_text = question_data.get('question', '')
        customer_type = question_data.get('customer_type', 'normal')
        complexity = question_data.get('complexity', 'medium')
        
        print(f"\n🔄 Processing question {question_id}: {question_text[:60]}...")
        
        try:
            if chatbot_agent:
                # Create state for the Chatbot Agent
                from datetime import datetime
                state = {
                    'query': question_text,
                    'user_id': 'test_user',
                    'session_id': f'test_session_{i}',
                    'query_id': f'query_{question_id}',
                    'timestamp': datetime.now().isoformat(),
                    'messages': []
                }
                
                # Process through agent using __call__ method
                response_state = chatbot_agent(state)
                
                answer = response_state.get('ai_response', 'No response generated')
                confidence = response_state.get('initial_assessment', {}).get('confidence', 0.8)
                
                # Use improved flag analysis
                flag_analysis = analyze_chatbot_response(answer, confidence)
                needs_escalation = flag_analysis['needs_escalation']
                needs_more_input = flag_analysis['needs_more_input']
                
            else:
                # Mock processing for demonstration
                import random
                answer = f"Thank you for your question about {question_text[:30]}... I'd be happy to help you with that. [This is a mock response for demonstration]"
                confidence = random.uniform(0.6, 0.95)
                
                # Use improved flag analysis even for mock responses
                flag_analysis = analyze_chatbot_response(answer, confidence)
                needs_escalation = flag_analysis['needs_escalation']
                needs_more_input = flag_analysis['needs_more_input']
            
            # Create result entry with enhanced analysis
            result = {
                'id': question_id,
                'original_question': question_text,
                'customer_type': customer_type,
                'complexity': complexity,
                'ai_answer': answer,
                'confidence_score': confidence,
                'needs_escalation': needs_escalation,
                'needs_more_input': needs_more_input,
                'escalation_reason': flag_analysis.get('escalation_reason'),
                'input_reason': flag_analysis.get('input_reason'),
                'question_count': flag_analysis.get('question_count', 0),
                'analysis_details': flag_analysis.get('analysis_details', {}),
                'processing_time': datetime.now().isoformat(),
                'model_used': current_preferred_model,
                'temperature': current_temperature,
                'config_source': 'temp_files' if using_temp_files else 'base_files'
            }
            
            results.append(result)
            
            # Show progress with enhanced status
            if needs_more_input:
                reason = flag_analysis.get('input_reason', 'unknown')
                status = f"🔄 Needs more input ({reason})"
            elif needs_escalation:
                reason = flag_analysis.get('escalation_reason', 'unknown')
                status = f"⚠️ Escalation needed ({reason})"
            else:
                status = "✅ Complete"
            
            print(f"   {status} (confidence: {confidence:.2f})")
            
        except Exception as e:
            print(f"   ❌ Error processing question: {e}")
            # Add error result
            result = {
                'id': question_id,
                'original_question': question_text,
                'customer_type': customer_type,
                'complexity': complexity,
                'ai_answer': f"Error processing question: {e}",
                'confidence_score': 0.0,
                'needs_escalation': True,
                'needs_more_input': False,
                'escalation_reason': 'processing_error',
                'input_reason': None,
                'question_count': 0,
                'analysis_details': {'error': True},
                'processing_time': datetime.now().isoformat(),
                'model_used': current_preferred_model,
                'temperature': current_temperature,
                'config_source': 'temp_files' if using_temp_files else 'base_files',
                'error': str(e)
            }
            results.append(result)
    
    print("\n" + "="*50)
    print(f"✅ Processing complete! Processed {len(results)} questions.")
    
    # Enhanced summary statistics
    total_questions = len(results)
    needs_more_input_count = sum(1 for r in results if r['needs_more_input'])
    needs_escalation_count = sum(1 for r in results if r['needs_escalation'])
    avg_confidence = sum(r['confidence_score'] for r in results) / total_questions if total_questions > 0 else 0
    
    # Breakdown by escalation reason
    escalation_reasons = {}
    input_reasons = {}
    for r in results:
        if r['needs_escalation'] and r['escalation_reason']:
            escalation_reasons[r['escalation_reason']] = escalation_reasons.get(r['escalation_reason'], 0) + 1
        if r['needs_more_input'] and r['input_reason']:
            input_reasons[r['input_reason']] = input_reasons.get(r['input_reason'], 0) + 1
    
    print(f"\n📊 Enhanced Summary:")
    print(f"  Total questions: {total_questions}")
    print(f"  Config source: {'Temporary files (with edits)' if using_temp_files else 'Base config files'}")
    print(f"  Need more input: {needs_more_input_count} ({needs_more_input_count/total_questions*100:.1f}%)")
    if input_reasons:
        print(f"    Reasons: {input_reasons}")
    print(f"  Need escalation: {needs_escalation_count} ({needs_escalation_count/total_questions*100:.1f}%)")
    if escalation_reasons:
        print(f"    Reasons: {escalation_reasons}")
    print(f"  Average confidence: {avg_confidence:.2f}")
    
    # Store config source info for export
    experiment_config_source = 'temp_files' if using_temp_files else 'base_files'

## Step 6: Complete Conversations with Customer AI

Continue conversations between customer AI and chatbot AI until natural resolution or escalation.

In [8]:
# Complete conversations with customer AI simulation
if 'results' in locals() and results:
    import random  # Add missing import
    
    def create_customer_ai_simulator(customer_type, complexity, model_name='anthropic_general_budget'):
        """Create a customer AI simulator based on customer profile"""
        
        # Define customer personas
        customer_personas = {
            'frustrated': {
                'style': 'Impatient, demanding, may use caps or exclamation points. Wants quick resolution.',
                'follow_up_tendency': 'high',
                'satisfaction_threshold': 0.8,
                'escalation_patience': 2  # Will demand escalation after 2 responses if not satisfied
            },
            'urgent': {
                'style': 'Time-sensitive, focused on immediate action. Professional but hurried.',
                'follow_up_tendency': 'high', 
                'satisfaction_threshold': 0.75,
                'escalation_patience': 3
            },
            'confused': {
                'style': 'Asks many clarifying questions, needs simple explanations. Polite but persistent.',
                'follow_up_tendency': 'very_high',
                'satisfaction_threshold': 0.7,
                'escalation_patience': 4
            },
            'normal': {
                'style': 'Professional, patient, reasonable expectations.',
                'follow_up_tendency': 'medium',
                'satisfaction_threshold': 0.65,
                'escalation_patience': 3
            }
        }
        
        # Define complexity behaviors
        complexity_behaviors = {
            'simple': {
                'satisfaction_boost': 0.1,  # Easier to satisfy
                'max_turns': 3,
                'question_depth': 'surface-level'
            },
            'medium': {
                'satisfaction_boost': 0.0,
                'max_turns': 5,
                'question_depth': 'moderate detail'
            },
            'complex': {
                'satisfaction_boost': -0.1,  # Harder to satisfy
                'max_turns': 7,
                'question_depth': 'detailed technical'
            }
        }
        
        persona = customer_personas.get(customer_type, customer_personas['normal'])
        behavior = complexity_behaviors.get(complexity, complexity_behaviors['medium'])
        
        class CustomerAISimulator:
            def __init__(self, customer_type, complexity):
                self.customer_type = customer_type  # Store customer type as instance variable
                self.complexity = complexity  # Store complexity as instance variable
                self.persona = persona
                self.behavior = behavior
                self.turn_count = 0
                self.satisfaction_level = 0.0
                self.conversation_history = []
                
                # Create LLM for customer simulation
                try:
                    provider_factory = LLMProviderFactory()
                    self.llm = provider_factory.create_provider(model_name)
                except Exception as e:
                    print(f"Warning: Could not create customer AI simulator LLM: {e}")
                    self.llm = None
            
            def generate_response(self, chatbot_response, original_question):
                """Generate customer response to chatbot"""
                self.turn_count += 1
                
                # Analyze satisfaction with chatbot response
                satisfaction_score = self._analyze_satisfaction(chatbot_response)
                self.satisfaction_level = satisfaction_score
                
                # Check if conversation should end
                is_satisfied = satisfaction_score >= (self.persona['satisfaction_threshold'] + self.behavior['satisfaction_boost'])
                is_escalating = self.turn_count >= self.persona['escalation_patience'] and satisfaction_score < 0.6
                is_max_turns = self.turn_count >= self.behavior['max_turns']
                
                if is_satisfied:
                    return self._generate_satisfaction_response()
                elif is_escalating:
                    return self._generate_escalation_request()
                elif is_max_turns:
                    return self._generate_final_response()
                else:
                    return self._generate_follow_up_question(chatbot_response, original_question)
            
            def _analyze_satisfaction(self, response):
                """Analyze how satisfied the customer would be with the response"""
                response_lower = response.lower()
                
                # Positive indicators
                positive_score = 0
                if any(phrase in response_lower for phrase in ['specifically', 'here\'s how', 'i can help', 'let me explain']):
                    positive_score += 0.2
                if any(phrase in response_lower for phrase in ['immediately', 'right away', 'quickly']):
                    positive_score += 0.15
                if len(response) > 100:  # Detailed response
                    positive_score += 0.1
                
                # Negative indicators  
                negative_score = 0
                if any(phrase in response_lower for phrase in ['i need more information', 'could you provide', 'what type']):
                    negative_score += 0.3  # Asking for more info is frustrating
                if any(phrase in response_lower for phrase in ['unfortunately', 'however', 'but']):
                    negative_score += 0.1
                if response.count('?') > 2:  # Too many questions back
                    negative_score += 0.2
                
                # Base satisfaction varies by customer type - use self.customer_type
                base_satisfaction = {
                    'frustrated': 0.3,
                    'urgent': 0.4, 
                    'confused': 0.5,
                    'normal': 0.6
                }.get(self.customer_type, 0.5)
                
                return max(0.0, min(1.0, base_satisfaction + positive_score - negative_score))
            
            def _generate_satisfaction_response(self):
                """Generate a satisfied customer response"""
                satisfied_responses = {
                    'frustrated': [
                        "Okay, that makes sense. Thanks for clearing that up.",
                        "Finally! Thank you for the explanation.",
                        "Alright, I understand now. That helps."
                    ],
                    'urgent': [
                        "Perfect, that's exactly what I needed to know. Thank you!",
                        "Great, I'll do that right away. Thanks for the quick help!",
                        "Excellent, that answers my question. Much appreciated."
                    ],
                    'confused': [
                        "Oh I see! That makes much more sense now. Thank you for explaining it so clearly.",
                        "Thank you! That explanation really helped me understand.",
                        "Perfect! Now I get it. I really appreciate your patience."
                    ],
                    'normal': [
                        "Thank you for the helpful information. That resolves my question.",
                        "Great, that's exactly what I needed to know. Thanks!",
                        "Perfect, I understand now. Thank you for your assistance."
                    ]
                }
                
                return random.choice(satisfied_responses.get(self.customer_type, satisfied_responses['normal']))
            
            def _generate_escalation_request(self):
                """Generate escalation request"""
                escalation_responses = {
                    'frustrated': [
                        "This isn't working. I need to speak to someone who can actually help me!",
                        "I'm getting nowhere with this. Transfer me to a supervisor NOW!",
                        "Enough! Get me a human who knows what they're doing!"
                    ],
                    'urgent': [
                        "I need this resolved immediately. Can you transfer me to someone who can handle this urgently?",
                        "Time is critical here. I need to speak with a specialist right away.",
                        "This is urgent - please connect me with someone who can resolve this now."
                    ],
                    'confused': [
                        "I'm still really confused. Could you please connect me with someone who can walk me through this step by step?",
                        "I don't think I'm understanding this correctly. Can I speak with someone who can explain this more simply?",
                        "I'm getting more confused. Could you transfer me to someone who specializes in helping customers like me?"
                    ],
                    'normal': [
                        "I think I need to speak with a specialist about this. Could you please transfer me?",
                        "This seems like it might require human expertise. Can you connect me with the right department?",
                        "I'd like to speak with someone who can provide more detailed assistance."
                    ]
                }
                
                return random.choice(escalation_responses.get(self.customer_type, escalation_responses['normal']))
            
            def _generate_final_response(self):
                """Generate final response when max turns reached"""
                final_responses = {
                    'frustrated': "Look, I've been going in circles here. Just transfer me to someone else.",
                    'urgent': "I've spent too much time on this already. I need to speak with a human.",
                    'confused': "I'm still not clear on this. I think I need to talk to someone in person.",
                    'normal': "I think this might be beyond what we can resolve here. Could you transfer me to the appropriate department?"
                }
                
                return final_responses.get(self.customer_type, final_responses['normal'])
            
            def _generate_follow_up_question(self, chatbot_response, original_question):
                """Generate intelligent follow-up question using AI if available"""
                
                if self.llm:
                    # Use AI to generate contextual follow-up - FIX: Use correct method signature
                    prompt = f'''You are a {self.customer_type} customer with a {self.complexity} question about insurance. 
                    
Your personality: {self.persona['style']}
Question complexity: {self.behavior['question_depth']}
Turn {self.turn_count} of conversation.

Original question: {original_question}
Chatbot's response: {chatbot_response}

Generate a follow-up response that a {self.customer_type} customer would realistically ask. Be specific to the chatbot's response and maintain the personality. Make it {self.behavior['question_depth']} in nature.

Respond as the customer would (2-3 sentences max):'''
                    
                    try:
                        # FIX: Use correct method signature (prompt, system_prompt)
                        response = self.llm.generate_response(prompt)
                        
                        if response and len(response.strip()) > 10:
                            return response.strip()
                    except Exception as e:
                        print(f"Warning: AI follow-up generation failed: {e}")
                
                # Fallback to template-based responses
                return self._generate_template_follow_up(chatbot_response, original_question)
            
            def _generate_template_follow_up(self, chatbot_response, original_question):
                """Generate template-based follow-up questions"""
                
                # Analyze what the chatbot asked for
                response_lower = chatbot_response.lower()
                
                if 'policy number' in response_lower:
                    return f"My policy number is POL-{random.randint(100000, 999999)}. Now what?"
                elif 'claim number' in response_lower:
                    return f"It's claim #{random.randint(10000, 99999)}. What's the next step?"
                elif 'type of coverage' in response_lower or 'which coverage' in response_lower:
                    coverage_types = ['comprehensive', 'collision', 'liability', 'homeowners', 'auto']
                    return f"I have {random.choice(coverage_types)} coverage. Does that help?"
                elif 'when did' in response_lower or 'what date' in response_lower:
                    return "This happened yesterday around 3 PM. What do I do now?"
                elif 'how much' in response_lower or 'what amount' in response_lower:
                    return f"It's about ${random.randint(500, 5000)} in damage. What's next?"
                elif 'documents' in response_lower or 'paperwork' in response_lower:
                    return "I have photos and a police report. How do I submit them?"
                else:
                    # Generic follow-ups based on customer type
                    generic_followups = {
                        'frustrated': "That doesn't really answer my question. Can you be more specific?",
                        'urgent': "Okay, but what do I do RIGHT NOW? This is time-sensitive!",
                        'confused': "I'm still not sure I understand. Can you explain it differently?",
                        'normal': "Could you provide more specific steps on what I should do next?"
                    }
                    return generic_followups.get(self.customer_type, generic_followups['normal'])
        
        return CustomerAISimulator(customer_type, complexity)  # Pass parameters to constructor
    
    def conduct_full_conversation(question_data, max_conversation_turns=8):
        """Conduct a full conversation between customer AI and chatbot until resolution"""
        
        question_id = question_data.get('id', 1)
        question_text = question_data.get('question', '')
        customer_type = question_data.get('customer_type', 'normal')
        complexity = question_data.get('complexity', 'medium')
        
        # Get the model from questions file or use current preferred model
        model_to_use = question_data.get('model', current_preferred_model)
        
        print(f"🗣️ Starting full conversation for question {question_id}")
        print(f"   Customer: {customer_type}, Complexity: {complexity}, Model: {model_to_use}")
        
        # Create customer AI simulator
        try:
            customer_ai = create_customer_ai_simulator(customer_type, complexity, model_to_use)
        except Exception as e:
            print(f"   ❌ Error creating customer AI: {e}")
            return {
                'id': question_id,
                'original_question': question_text,
                'customer_type': customer_type,
                'complexity': complexity,
                'error': f"Failed to create customer AI: {e}",
                'conversation_complete': False,
                'final_outcome': 'error'
            }
        
        # Initialize conversation
        conversation_history = []
        current_query = question_text
        turn_number = 1
        
        # Conversation loop
        while turn_number <= max_conversation_turns:
            print(f"   Turn {turn_number}: Processing...")
            
            try:
                if chatbot_agent:
                    # Create state for chatbot
                    state = {
                        'query': current_query,
                        'user_id': 'test_user',
                        'session_id': f'conversation_session_{question_id}',
                        'query_id': f'query_{question_id}_turn_{turn_number}',
                        'timestamp': datetime.now().isoformat(),
                        'messages': []  # Start with empty messages for each turn
                    }
                    
                    # Get chatbot response
                    response_state = chatbot_agent(state)
                    chatbot_response = response_state.get('ai_response', 'No response generated')
                    confidence = response_state.get('initial_assessment', {}).get('confidence', 0.8)
                    
                else:
                    # Mock chatbot response
                    chatbot_response = f"Thank you for your question. Let me help you with that... [Mock response turn {turn_number}]"
                    confidence = 0.8
                
                # Add to conversation history
                conversation_turn = {
                    'turn_number': turn_number,
                    'customer_query': current_query,
                    'chatbot_response': chatbot_response,
                    'confidence': confidence,
                    'timestamp': datetime.now().isoformat()
                }
                conversation_history.append(conversation_turn)
                
                # Analyze chatbot response for escalation
                response_analysis = analyze_chatbot_response(chatbot_response, confidence)
                
                # Check if chatbot is escalating
                if response_analysis['needs_escalation']:
                    print(f"   🔄 Chatbot escalating: {response_analysis['escalation_reason']}")
                    conversation_turn['chatbot_action'] = 'escalation'
                    conversation_turn['escalation_reason'] = response_analysis['escalation_reason']
                    break
                
                # Generate customer response using AI simulator
                try:
                    customer_response = customer_ai.generate_response(chatbot_response, question_text)
                except Exception as e:
                    print(f"   ❌ Error generating customer response: {e}")
                    customer_response = "I'm having trouble understanding this. Can you help me differently?"
                
                # Add customer response to turn
                conversation_turn['customer_response'] = customer_response
                conversation_turn['customer_satisfaction'] = customer_ai.satisfaction_level
                
                # Check if customer is satisfied (conversation ends)
                if any(phrase in customer_response.lower() for phrase in ['thank you', 'that helps', 'perfect', 'great', 'excellent', 'makes sense']):
                    if customer_ai.satisfaction_level >= customer_ai.persona['satisfaction_threshold']:
                        print(f"   ✅ Customer satisfied!")
                        conversation_turn['customer_action'] = 'satisfied'
                        break
                
                # Check if customer is requesting escalation
                if any(phrase in customer_response.lower() for phrase in ['transfer', 'supervisor', 'specialist', 'human', 'someone else']):
                    print(f"   ⬆️ Customer requesting escalation")
                    conversation_turn['customer_action'] = 'escalation_request'
                    break
                
                # Prepare for next turn
                current_query = customer_response
                turn_number += 1
                
            except Exception as e:
                print(f"   ❌ Error in turn {turn_number}: {e}")
                conversation_turn = {
                    'turn_number': turn_number,
                    'customer_query': current_query,
                    'error': str(e),
                    'timestamp': datetime.now().isoformat()
                }
                conversation_history.append(conversation_turn)
                break
        
        # Determine final outcome
        final_turn = conversation_history[-1] if conversation_history else {}
        final_outcome = final_turn.get('customer_action', final_turn.get('chatbot_action', 'conversation_incomplete'))
        
        conversation_result = {
            'id': question_id,
            'original_question': question_text,
            'customer_type': customer_type,
            'complexity': complexity,
            'model_used': model_to_use,
            'conversation_history': conversation_history,
            'total_turns': len(conversation_history),
            'final_outcome': final_outcome,
            'customer_satisfaction': customer_ai.satisfaction_level if customer_ai else 0.0,
            'processing_time': datetime.now().isoformat(),
            'conversation_complete': final_outcome in ['satisfied', 'escalation', 'escalation_request']
        }
        
        print(f"   📋 Conversation complete: {final_outcome} in {len(conversation_history)} turns")
        return conversation_result
    
    # Process conversations for all questions
    print(f"🤖 Starting full conversations for {len(test_questions)} questions...")
    print(f"This will simulate realistic customer-chatbot interactions until resolution.\n")
    
    conversation_results = []
    
    for i, question_data in enumerate(test_questions):
        try:
            conversation_result = conduct_full_conversation(question_data)
            conversation_results.append(conversation_result)
            
            # Show progress
            outcome = conversation_result['final_outcome']
            turns = conversation_result['total_turns']
            satisfaction = conversation_result.get('customer_satisfaction', 0)
            
            print(f"Question {question_data['id']}: {outcome} ({turns} turns, satisfaction: {satisfaction:.2f})")
            
        except Exception as e:
            print(f"❌ Error processing conversation {question_data['id']}: {e}")
            # Add error result
            error_result = {
                'id': question_data['id'],
                'original_question': question_data.get('question', ''),
                'customer_type': question_data.get('customer_type', 'normal'),
                'complexity': question_data.get('complexity', 'medium'),
                'error': str(e),
                'conversation_complete': False,
                'final_outcome': 'error'
            }
            conversation_results.append(error_result)
    
    print(f"\n✅ All conversations completed!")
    
    # Update results with conversation data
    results = conversation_results
    
    # Summary statistics
    completed_conversations = [r for r in results if r.get('conversation_complete', False)]
    satisfied_customers = [r for r in results if r.get('final_outcome') == 'satisfied']
    escalated_conversations = [r for r in results if 'escalation' in r.get('final_outcome', '')]
    
    avg_turns = sum(r.get('total_turns', 0) for r in completed_conversations) / len(completed_conversations) if completed_conversations else 0
    avg_satisfaction = sum(r.get('customer_satisfaction', 0) for r in completed_conversations) / len(completed_conversations) if completed_conversations else 0
    
    print(f"\n📊 Conversation Summary:")
    print(f"  Completed conversations: {len(completed_conversations)}/{len(results)}")
    print(f"  Customer satisfaction: {len(satisfied_customers)} ({len(satisfied_customers)/len(results)*100:.1f}%)")
    print(f"  Escalations: {len(escalated_conversations)} ({len(escalated_conversations)/len(results)*100:.1f}%)")
    print(f"  Average turns per conversation: {avg_turns:.1f}")
    print(f"  Average customer satisfaction: {avg_satisfaction:.2f}")

else:
    print("⚠️ No questions loaded. Please load questions first.")

🤖 Starting full conversations for 20 questions...
This will simulate realistic customer-chatbot interactions until resolution.

🗣️ Starting full conversation for question 1
   Customer: frustrated, Complexity: medium, Model: local_general_standard
✅ Creating LLM provider: local_general_standard → llama-7b (llama)
✅ 12:29:07.143 [INFO    ] unknown         | Initializing LLM provider | model_name=unknown


llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_context: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized


✅ 12:29:10.975 [INFO    ] unknown         | LLM provider initialized successfully | model_name=unknown
   Turn 1: Processing...
✅ 12:29:29.533 [INFO    ] unknown         | Model call: unknown - generate_response | model_name=unknown operation=generate_response
✅ 12:29:44.493 [INFO    ] unknown         | Model call: unknown - generate_response | model_name=unknown operation=generate_response
   Turn 2: Processing...
✅ 12:29:49.085 [INFO    ] unknown         | Model call: unknown - generate_response | model_name=unknown operation=generate_response
   ⬆️ Customer requesting escalation
   📋 Conversation complete: escalation_request in 2 turns
Question 1: escalation_request (2 turns, satisfaction: 0.30)
🗣️ Starting full conversation for question 2
   Customer: urgent, Complexity: simple, Model: local_general_standard
✅ Creating LLM provider: local_general_standard → llama-7b (llama)
✅ 12:29:49.313 [INFO    ] unknown         | Initializing LLM provider | model_name=unknown


llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_context: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized


✅ 12:29:52.886 [INFO    ] unknown         | LLM provider initialized successfully | model_name=unknown
   Turn 1: Processing...
✅ 12:30:07.887 [INFO    ] unknown         | Model call: unknown - generate_response | model_name=unknown operation=generate_response
✅ 12:30:20.175 [INFO    ] unknown         | Model call: unknown - generate_response | model_name=unknown operation=generate_response
   Turn 2: Processing...
✅ 12:30:36.038 [INFO    ] unknown         | Model call: unknown - generate_response | model_name=unknown operation=generate_response
✅ 12:30:48.756 [INFO    ] unknown         | Model call: unknown - generate_response | model_name=unknown operation=generate_response
   Turn 3: Processing...
✅ 12:31:10.681 [INFO    ] unknown         | Model call: unknown - generate_response | model_name=unknown operation=generate_response
   Turn 4: Processing...
✅ 12:31:20.371 [INFO    ] unknown         | Model call: unknown - generate_response | model_name=unknown operation=generate_response

...

✅ 12:48:01.386 [INFO    ] unknown         | LLM provider initialized successfully | model_name=unknown
   Turn 1: Processing...
✅ 12:48:25.900 [INFO    ] unknown         | Model call: unknown - generate_response | model_name=unknown operation=generate_response
✅ 12:48:52.637 [INFO    ] unknown         | Model call: unknown - generate_response | model_name=unknown operation=generate_response
   Turn 2: Processing...
✅ 12:49:01.031 [INFO    ] unknown         | Model call: unknown - generate_response | model_name=unknown operation=generate_response
✅ 12:49:10.262 [INFO    ] unknown         | Model call: unknown - generate_response | model_name=unknown operation=generate_response
   Turn 3: Processing...
✅ 12:49:17.121 [INFO    ] unknown         | Model call: unknown - generate_response | model_name=unknown operation=generate_response
   ⬆️ Customer requesting escalation
   📋 Conversation complete: escalation_request in 3 turns
Question 18: escalation_request (3 turns, satisfaction: 0.40)


llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_context: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized


✅ 12:49:18.811 [INFO    ] unknown         | LLM provider initialized successfully | model_name=unknown
   Turn 1: Processing...
✅ 12:49:31.383 [INFO    ] unknown         | Model call: unknown - generate_response | model_name=unknown operation=generate_response
✅ 12:49:48.917 [INFO    ] unknown         | Model call: unknown - generate_response | model_name=unknown operation=generate_response
   Turn 2: Processing...
✅ 12:50:04.979 [INFO    ] unknown         | Model call: unknown - generate_response | model_name=unknown operation=generate_response
   ⬆️ Customer requesting escalation
   📋 Conversation complete: escalation_request in 2 turns
Question 19: escalation_request (2 turns, satisfaction: 0.40)
🗣️ Starting full conversation for question 20
   Customer: normal, Complexity: simple, Model: local_general_standard
✅ Creating LLM provider: local_general_standard → llama-7b (llama)
✅ 12:50:05.206 [INFO    ] unknown         | Initializing LLM provider | model_name=unknown


llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_context: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized


✅ 12:50:06.758 [INFO    ] unknown         | LLM provider initialized successfully | model_name=unknown
   Turn 1: Processing...
✅ 12:50:15.205 [INFO    ] unknown         | Model call: unknown - generate_response | model_name=unknown operation=generate_response
✅ 12:50:28.081 [INFO    ] unknown         | Model call: unknown - generate_response | model_name=unknown operation=generate_response
   ✅ Customer satisfied!
   📋 Conversation complete: satisfied in 1 turns
Question 20: satisfied (1 turns, satisfaction: 0.70)

✅ All conversations completed!

📊 Conversation Summary:
  Completed conversations: 20/20
  Customer satisfaction: 8 (40.0%)
  Escalations: 12 (60.0%)
  Average turns per conversation: 2.7
  Average customer satisfaction: 0.59


## Step 7: Export Results and Settings

Save the results and configuration settings to files with timestamps.

In [9]:
# Export conversation results and settings in structured YAML format
if 'results' in locals() and results:
    # Create timestamp for filenames
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    
    # Create experiment_runs directory if it doesn't exist
    experiment_dir = Path('/workspace/notebooks/experiment_runs')
    experiment_dir.mkdir(exist_ok=True)
    
    # Load current config contents from temp files if they exist
    def get_config_contents_for_export():
        """Get current config file contents for export"""
        if all(path.exists() for path in temp_file_paths.values()):
            print("📁 Exporting with temporary config file contents...")
            
            with open(temp_file_paths['agent_config'], 'r') as f:
                agent_config_content = f.read()
            with open(temp_file_paths['prompts_config'], 'r') as f:
                prompts_config_content = f.read()
            with open(temp_file_paths['models_config'], 'r') as f:
                models_config_content = f.read()
            
            return agent_config_content, prompts_config_content, models_config_content, True
        else:
            print("📁 Exporting with base config file contents...")
            
            with open(agent_config_path / 'config.yaml', 'r') as f:
                agent_config_content = f.read()
            with open(agent_config_path / 'prompts.yaml', 'r') as f:
                prompts_config_content = f.read()
            with open(agent_config_path / 'models.yaml', 'r') as f:
                models_config_content = f.read()
            
            return agent_config_content, prompts_config_content, models_config_content, False
    
    # Get config contents for export
    agent_config_content, prompts_config_content, models_config_content, used_temp_configs = get_config_contents_for_export()
    
    # Export results with full conversation data (JSON format for complex data)
    results_filename = f"chatbot_agent_output_{timestamp}.json"
    results_path = experiment_dir / results_filename
    
    # Create comprehensive export with conversation analysis
    export_data = {
        'experiment_metadata': {
            'timestamp': timestamp,
            'agent_type': 'chatbot_agent',
            'experiment_type': 'full_conversation_simulation',
            'total_questions': len(results),
            'completed_conversations': len([r for r in results if r.get('conversation_complete', False)]),
            'generation_model': current_preferred_model,
            'generation_timestamp': timestamp,
            'config_source': experiment_config_source
        },
        'conversation_results': results,
        'summary_statistics': {
            'total_conversations': len(results),
            'completed_conversations': len([r for r in results if r.get('conversation_complete', False)]),
            'satisfied_customers': len([r for r in results if r.get('final_outcome') == 'satisfied']),
            'escalated_conversations': len([r for r in results if 'escalation' in r.get('final_outcome', '')]),
            'error_conversations': len([r for r in results if r.get('final_outcome') == 'error']),
            'average_turns': sum(r.get('total_turns', 0) for r in results) / len(results) if results else 0,
            'average_satisfaction': sum(r.get('customer_satisfaction', 0) for r in results) / len(results) if results else 0,
            'satisfaction_rate': len([r for r in results if r.get('final_outcome') == 'satisfied']) / len(results) * 100 if results else 0,
            'escalation_rate': len([r for r in results if 'escalation' in r.get('final_outcome', '')]) / len(results) * 100 if results else 0
        },
        'conversation_analysis': {
            'by_customer_type': {},
            'by_complexity': {},
            'by_outcome': {}
        }
    }
    
    # Analyze by customer type
    customer_types = list(set(r.get('customer_type', 'unknown') for r in results))
    for ctype in customer_types:
        ctype_results = [r for r in results if r.get('customer_type') == ctype]
        export_data['conversation_analysis']['by_customer_type'][ctype] = {
            'count': len(ctype_results),
            'satisfaction_rate': len([r for r in ctype_results if r.get('final_outcome') == 'satisfied']) / len(ctype_results) * 100 if ctype_results else 0,
            'escalation_rate': len([r for r in ctype_results if 'escalation' in r.get('final_outcome', '')]) / len(ctype_results) * 100 if ctype_results else 0,
            'average_turns': sum(r.get('total_turns', 0) for r in ctype_results) / len(ctype_results) if ctype_results else 0,
            'average_satisfaction': sum(r.get('customer_satisfaction', 0) for r in ctype_results) / len(ctype_results) if ctype_results else 0
        }
    
    # Analyze by complexity
    complexities = list(set(r.get('complexity', 'unknown') for r in results))
    for complexity in complexities:
        complexity_results = [r for r in results if r.get('complexity') == complexity]
        export_data['conversation_analysis']['by_complexity'][complexity] = {
            'count': len(complexity_results),
            'satisfaction_rate': len([r for r in complexity_results if r.get('final_outcome') == 'satisfied']) / len(complexity_results) * 100 if complexity_results else 0,
            'escalation_rate': len([r for r in complexity_results if 'escalation' in r.get('final_outcome', '')]) / len(complexity_results) * 100 if complexity_results else 0,
            'average_turns': sum(r.get('total_turns', 0) for r in complexity_results) / len(complexity_results) if complexity_results else 0,
            'average_satisfaction': sum(r.get('customer_satisfaction', 0) for r in complexity_results) / len(complexity_results) if complexity_results else 0
        }
    
    # Analyze by outcome
    outcomes = list(set(r.get('final_outcome', 'unknown') for r in results))
    for outcome in outcomes:
        outcome_results = [r for r in results if r.get('final_outcome') == outcome]
        export_data['conversation_analysis']['by_outcome'][outcome] = {
            'count': len(outcome_results),
            'percentage': len(outcome_results) / len(results) * 100 if results else 0,
            'average_turns': sum(r.get('total_turns', 0) for r in outcome_results) / len(outcome_results) if outcome_results else 0,
            'average_satisfaction': sum(r.get('customer_satisfaction', 0) for r in outcome_results) / len(outcome_results) if outcome_results else 0
        }
    
    with open(results_path, 'w') as f:
        json.dump(export_data, f, indent=2)
    
    print(f"📄 Conversation results exported to: {results_path}")
    
    # NEW: Export structured YAML experiment settings file
    settings_filename = f"chatbot_agent_settings_{timestamp}.yaml"
    settings_path = experiment_dir / settings_filename
    
    # Get uploaded filename safely
    uploaded_filename = 'unknown'
    if file_upload.value:
        if isinstance(file_upload.value, tuple) and len(file_upload.value) > 0:
            uploaded_filename = file_upload.value[0]['name']
        elif isinstance(file_upload.value, dict) and len(file_upload.value) > 0:
            uploaded_filename = list(file_upload.value.values())[0]['metadata']['name']
    
    # Create structured YAML export
    yaml_experiment_data = {
        'experiment': {
            'id': f"chatbot_agent_{timestamp}",
            'timestamp': timestamp,
            'agent_type': 'chatbot_agent',
            'experiment_type': 'full_conversation_simulation',
            'config_source': experiment_config_source,
            'input_file': uploaded_filename,
            'total_questions': len(results),
            'completed_conversations': len([r for r in results if r.get('conversation_complete', False)])
        },
        'base_configs': {
            'agent_config': agent_config_content,
            'prompts_config': prompts_config_content,
            'models_config': models_config_content
        }
    }
    
    # Write YAML file with literal block scalars to preserve formatting
    yaml_content = f"""# Chatbot Agent Experiment Settings
# Generated: {timestamp}
# Config source: {experiment_config_source}

experiment:
  id: "chatbot_agent_{timestamp}"
  timestamp: "{timestamp}"
  agent_type: "chatbot_agent"
  experiment_type: "full_conversation_simulation"
  config_source: "{experiment_config_source}"
  input_file: "{uploaded_filename}"
  total_questions: {len(results)}
  completed_conversations: {len([r for r in results if r.get('conversation_complete', False)])}

base_configs:
  agent_config: |
{yaml.dump(yaml.safe_load(agent_config_content), default_flow_style=False, sort_keys=False).rstrip()}

  prompts_config: |
{yaml.dump(yaml.safe_load(prompts_config_content), default_flow_style=False, sort_keys=False).rstrip()}

  models_config: |
{yaml.dump(yaml.safe_load(models_config_content), default_flow_style=False, sort_keys=False).rstrip()}
"""
    
    with open(settings_path, 'w') as f:
        f.write(yaml_content)
    
    print(f"⚙️ Structured YAML settings exported to: {settings_path}")
    
    # Show enhanced summary
    stats = export_data['summary_statistics']
    print(f"\n📊 Export Summary:")
    print(f"  Timestamp: {timestamp}")
    print(f"  Results file: {results_filename} (JSON)")
    print(f"  Settings file: {settings_filename} (YAML)")
    print(f"  Config source: {experiment_config_source}")
    print(f"  Total conversations: {stats['total_conversations']}")
    print(f"  Completed conversations: {stats['completed_conversations']}")
    print(f"  Satisfaction rate: {stats['satisfaction_rate']:.1f}%")
    print(f"  Escalation rate: {stats['escalation_rate']:.1f}%")
    print(f"  Average turns per conversation: {stats['average_turns']:.1f}")
    print(f"  Average customer satisfaction: {stats['average_satisfaction']:.2f}")
    print(f"  Files saved to: {experiment_dir.absolute()}")
    
    print(f"\n📋 YAML Structure for Import:")
    print(f"  experiment.id: chatbot_agent_{timestamp}")
    print(f"  base_configs.agent_config: Complete agent configuration")
    print(f"  base_configs.prompts_config: Complete prompts configuration")
    print(f"  base_configs.models_config: Complete models configuration")
    print(f"  💡 Use this YAML file to recreate exact experiment conditions")
    
else:
    print("⚠️ No conversation results to export. Please complete conversations first.")

📁 Exporting with temporary config file contents...
📄 Conversation results exported to: /workspace/notebooks/experiment_runs/chatbot_agent_output_20250720_1250.json
⚙️ Structured YAML settings exported to: /workspace/notebooks/experiment_runs/chatbot_agent_settings_20250720_1250.yaml

📊 Export Summary:
  Timestamp: 20250720_1250
  Results file: chatbot_agent_output_20250720_1250.json (JSON)
  Settings file: chatbot_agent_settings_20250720_1250.yaml (YAML)
  Config source: temp_files
  Total conversations: 20
  Completed conversations: 20
  Satisfaction rate: 40.0%
  Escalation rate: 60.0%
  Average turns per conversation: 2.7
  Average customer satisfaction: 0.59
  Files saved to: /workspace/notebooks/experiment_runs

📋 YAML Structure for Import:
  experiment.id: chatbot_agent_20250720_1250
  base_configs.agent_config: Complete agent configuration
  base_configs.prompts_config: Complete prompts configuration
  base_configs.models_config: Complete models configuration
  💡 Use this YAML f

## Step 8: Review Results

Display and analyze the final results.

In [10]:
# Review and analyze conversation results
if 'results' in locals() and results:
    print("📋 Conversation Results Review and Analysis\n")
    
    # Create DataFrame for analysis
    df_results = pd.DataFrame([
        {
            'id': r.get('id'),
            'customer_type': r.get('customer_type'),
            'complexity': r.get('complexity'),
            'total_turns': r.get('total_turns', 0),
            'final_outcome': r.get('final_outcome'),
            'customer_satisfaction': r.get('customer_satisfaction', 0),
            'conversation_complete': r.get('conversation_complete', False)
        }
        for r in results
    ])
    
    print("=== CONVERSATION SUMMARY STATISTICS ===")
    print(f"Total conversations: {len(df_results)}")
    print(f"Completed conversations: {df_results['conversation_complete'].sum()}")
    print(f"Average turns per conversation: {df_results['total_turns'].mean():.1f}")
    print(f"Average customer satisfaction: {df_results['customer_satisfaction'].mean():.3f}")
    
    # Outcome distribution
    print("\n=== CONVERSATION OUTCOMES ===")
    outcome_counts = df_results['final_outcome'].value_counts()
    for outcome, count in outcome_counts.items():
        percentage = count / len(df_results) * 100
        print(f"{outcome}: {count} ({percentage:.1f}%)")
    
    # Analysis by customer type
    print("\n=== ANALYSIS BY CUSTOMER TYPE ===")
    customer_analysis = df_results.groupby('customer_type').agg({
        'total_turns': 'mean',
        'customer_satisfaction': 'mean',
        'conversation_complete': 'sum'
    }).round(3)
    customer_analysis.columns = ['Avg Turns', 'Avg Satisfaction', 'Completed']
    print(customer_analysis)
    
    # Satisfaction rate by customer type
    print("\nSatisfaction Rate by Customer Type:")
    for ctype in df_results['customer_type'].unique():
        ctype_data = df_results[df_results['customer_type'] == ctype]
        satisfied = len(ctype_data[ctype_data['final_outcome'] == 'satisfied'])
        total = len(ctype_data)
        print(f"  {ctype}: {satisfied}/{total} ({satisfied/total*100:.1f}%)")
    
    # Analysis by complexity
    print("\n=== ANALYSIS BY COMPLEXITY ===")
    complexity_analysis = df_results.groupby('complexity').agg({
        'total_turns': 'mean',
        'customer_satisfaction': 'mean',
        'conversation_complete': 'sum'
    }).round(3)
    complexity_analysis.columns = ['Avg Turns', 'Avg Satisfaction', 'Completed']
    print(complexity_analysis)
    
    # Show detailed results table
    print("\n=== CONVERSATION SUMMARY TABLE ===")
    display_df = df_results[['id', 'customer_type', 'complexity', 'total_turns', 'final_outcome', 'customer_satisfaction']].copy()
    display_df['customer_satisfaction'] = display_df['customer_satisfaction'].round(3)
    display(display_df)
    
    # Show sample full conversations
    print("\n=== SAMPLE FULL CONVERSATIONS ===")
    
    # Show 2 satisfied and 2 escalated conversations for analysis
    satisfied_conversations = [r for r in results if r.get('final_outcome') == 'satisfied']
    escalated_conversations = [r for r in results if 'escalation' in r.get('final_outcome', '')]
    
    def display_conversation(conversation_data, max_turns=3):
        """Display a conversation with turn-by-turn analysis"""
        conv_id = conversation_data.get('id')
        customer_type = conversation_data.get('customer_type')
        complexity = conversation_data.get('complexity')
        outcome = conversation_data.get('final_outcome')
        satisfaction = conversation_data.get('customer_satisfaction', 0)
        
        print(f"\n--- Conversation {conv_id} ({customer_type}, {complexity}) ---")
        print(f"Original Question: {conversation_data.get('original_question', '')}")
        print(f"Final Outcome: {outcome} (Customer Satisfaction: {satisfaction:.2f})")
        
        conversation_history = conversation_data.get('conversation_history', [])
        turns_to_show = min(max_turns, len(conversation_history))
        
        for i, turn in enumerate(conversation_history[:turns_to_show]):
            turn_num = turn.get('turn_number', i+1)
            print(f"\n  Turn {turn_num}:")
            print(f"    Customer: {turn.get('customer_query', '')[:150]}{'...' if len(turn.get('customer_query', '')) > 150 else ''}")
            print(f"    Chatbot:  {turn.get('chatbot_response', '')[:150]}{'...' if len(turn.get('chatbot_response', '')) > 150 else ''}")
            
            if 'customer_response' in turn:
                print(f"    Customer Reply: {turn.get('customer_response', '')[:150]}{'...' if len(turn.get('customer_response', '')) > 150 else ''}")
                print(f"    Satisfaction: {turn.get('customer_satisfaction', 0):.2f}")
        
        if len(conversation_history) > turns_to_show:
            print(f"  ... and {len(conversation_history) - turns_to_show} more turns")
    
    # Show satisfied conversations
    if satisfied_conversations:
        print("\n🟢 SATISFIED CUSTOMER CONVERSATIONS:")
        for conv in satisfied_conversations[:2]:
            display_conversation(conv)
    
    # Show escalated conversations  
    if escalated_conversations:
        print("\n🔴 ESCALATED CONVERSATIONS:")
        for conv in escalated_conversations[:2]:
            display_conversation(conv)
    
    # Conversation insights
    print("\n=== CONVERSATION INSIGHTS ===")
    
    # Turn analysis
    short_conversations = len(df_results[df_results['total_turns'] <= 2])
    medium_conversations = len(df_results[(df_results['total_turns'] > 2) & (df_results['total_turns'] <= 5)])
    long_conversations = len(df_results[df_results['total_turns'] > 5])
    
    print(f"Conversation Length Distribution:")
    print(f"  Short (1-2 turns): {short_conversations} ({short_conversations/len(df_results)*100:.1f}%)")
    print(f"  Medium (3-5 turns): {medium_conversations} ({medium_conversations/len(df_results)*100:.1f}%)")
    print(f"  Long (6+ turns): {long_conversations} ({long_conversations/len(df_results)*100:.1f}%)")
    
    # Satisfaction insights
    high_satisfaction = len(df_results[df_results['customer_satisfaction'] >= 0.8])
    medium_satisfaction = len(df_results[(df_results['customer_satisfaction'] >= 0.5) & (df_results['customer_satisfaction'] < 0.8)])
    low_satisfaction = len(df_results[df_results['customer_satisfaction'] < 0.5])
    
    print(f"\nCustomer Satisfaction Distribution:")
    print(f"  High (0.8+): {high_satisfaction} ({high_satisfaction/len(df_results)*100:.1f}%)")
    print(f"  Medium (0.5-0.8): {medium_satisfaction} ({medium_satisfaction/len(df_results)*100:.1f}%)")
    print(f"  Low (<0.5): {low_satisfaction} ({low_satisfaction/len(df_results)*100:.1f}%)")
    
    print("\n" + "="*80)
    print("✅ Conversation analysis complete! Check the exported files for full conversation details.")
    print("💡 Each conversation includes turn-by-turn customer-AI interaction data for detailed analysis.")
    
else:
    print("⚠️ No conversation results to review. Please complete conversations first.")

📋 Conversation Results Review and Analysis

=== CONVERSATION SUMMARY STATISTICS ===
Total conversations: 20
Completed conversations: 20
Average turns per conversation: 2.7
Average customer satisfaction: 0.590

=== CONVERSATION OUTCOMES ===
escalation_request: 12 (60.0%)
satisfied: 8 (40.0%)

=== ANALYSIS BY CUSTOMER TYPE ===
               Avg Turns  Avg Satisfaction  Completed
customer_type                                        
confused             3.4              0.71          5
frustrated           1.8              0.41          5
normal               1.8              0.70          5
urgent               3.8              0.54          5

Satisfaction Rate by Customer Type:
  frustrated: 0/5 (0.0%)
  urgent: 0/5 (0.0%)
  confused: 4/5 (80.0%)
  normal: 4/5 (80.0%)

=== ANALYSIS BY COMPLEXITY ===
            Avg Turns  Avg Satisfaction  Completed
complexity                                        
complex         3.000             0.575          6
medium          3.000             0

,id,customer_type,complexity,total_turns,final_outcome,customer_satisfaction
0,1,frustrated,medium,2,escalation_request,0.30
1,2,urgent,simple,4,escalation_request,0.65
2,3,confused,simple,1,satisfied,0.80
3,4,normal,simple,3,escalation_request,0.70
4,5,frustrated,complex,1,escalation_request,0.55
5,6,confused,medium,3,satisfied,0.75
6,7,urgent,complex,4,escalation_request,0.50
7,8,urgent,simple,4,escalation_request,0.65
8,9,normal,simple,1,satisfied,0.70
9,10,confused,medium,6,escalation_request,0.50



=== SAMPLE FULL CONVERSATIONS ===

🟢 SATISFIED CUSTOMER CONVERSATIONS:

--- Conversation 3 (confused, simple) ---
Original Question: Can someone explain what a deductible is? I keep seeing this word but don't understand.
Final Outcome: satisfied (Customer Satisfaction: 0.80)

  Turn 1:
    Customer: Can someone explain what a deductible is? I keep seeing this word but don't understand.
    Chatbot:  
- Use simple language and avoid jargon

AGENT RESPONSE: Hello! I'd be happy to help you understand what a deductible is. A deductible is an amount of...
    Customer Reply: Oh I see! That makes much more sense now. Thank you for explaining it so clearly.
    Satisfaction: 0.80

--- Conversation 6 (confused, medium) ---
Original Question: What's the difference between comprehensive and collision coverage? I'm trying to decide what I need.
Final Outcome: satisfied (Customer Satisfaction: 0.75)

  Turn 1:
    Customer: What's the difference between comprehensive and collision coverage? I'm t

## Next Steps

🎉 **Congratulations!** You've successfully tested the Chatbot Agent with full conversation simulation.

### What you've accomplished:
- ✅ Loaded and customized Chatbot Agent configuration
- ✅ Loaded test questions from file with customer profiles  
- ✅ Conducted full conversations between customer AI and chatbot AI
- ✅ Simulated realistic customer interactions until resolution or escalation
- ✅ Captured complete conversation flows with satisfaction tracking
- ✅ Exported comprehensive results with conversation analysis
- ✅ Analyzed performance by customer type, complexity, and outcomes

### Your exported files contain:
- **Conversation Results**: Complete turn-by-turn interaction data including:
  - Full conversation history for each question
  - Customer satisfaction scores at each turn
  - Final outcomes (satisfied, escalation, etc.)
  - Detailed conversation analysis by customer type and complexity
- **Settings file**: Complete configuration used for this experiment
- **Conversation Analysis**: Statistical breakdown of conversation patterns

### Key Features of the Conversation System:
- **Customer AI Simulation**: Realistic customer personas (frustrated, urgent, confused, normal)
- **Dynamic Satisfaction Tracking**: AI-powered customer satisfaction scoring
- **Natural Conversation Flow**: Continues until natural resolution or escalation
- **Personality-Based Responses**: Customer responses match their personality type
- **Escalation Detection**: Automatic detection when customers or chatbot need human assistance
- **Model Integration**: Uses actual models from questions file for both customer and chatbot

### Conversation Outcomes Explained:
- **Satisfied**: Customer received satisfactory answer and conversation ended positively
- **Escalation**: Chatbot determined the query needed human specialist assistance  
- **Escalation Request**: Customer explicitly requested to speak with a human
- **Conversation Incomplete**: Reached maximum turns without resolution

### Ready for Advanced Analysis?
- **Conversation Patterns**: Analyze which question types lead to longer conversations
- **Satisfaction Drivers**: Identify what chatbot responses increase customer satisfaction
- **Escalation Triggers**: Understand when and why conversations need human intervention
- **Customer Journey Mapping**: Track how different customer types interact with the system
- **Model Performance**: Compare how different AI models handle customer interactions

### To optimize your chatbot:
1. **Review Escalated Conversations**: Look for patterns in conversations that required escalation
2. **Analyze Low Satisfaction**: Identify what causes customer dissatisfaction
3. **Study Successful Resolutions**: Learn from conversations that ended with satisfied customers
4. **Adjust Configuration**: Modify agent settings based on conversation insights
5. **Test Iteratively**: Run multiple experiments with different configurations

### Workflow for Continuous Improvement:
1. **Generate Questions**: Use `question_generator.ipynb` to create diverse test scenarios
2. **Run Conversations**: Use this notebook to conduct full conversation simulations
3. **Analyze Results**: Review conversation patterns and customer satisfaction metrics
4. **Optimize Configuration**: Adjust chatbot settings based on insights
5. **Validate Changes**: Re-run tests to confirm improvements
6. **Deploy**: Apply successful configurations to production systems

### Advanced Features Available:
- **Multi-Model Testing**: Test different AI models for customer and chatbot roles
- **Custom Customer Personas**: Modify customer AI behavior for specific use cases  
- **Conversation Length Controls**: Adjust maximum turns and escalation thresholds
- **Satisfaction Threshold Tuning**: Customize what constitutes a satisfied customer
- **Real-time Analysis**: Monitor conversation quality as it happens

---
*This notebook simulates realistic customer service interactions to optimize AI chatbot performance and improve customer satisfaction outcomes.*